Step 3 - Climate Analysis and Exploration

You are now ready to use Python and SQLAlchemy to do basic climate analysis and data exploration on your new weather station tables. All of the following analysis should be completed using SQLAlchemy ORM queries, Pandas, and Matplotlib.


Create a Jupyter Notebook file called climate_analysis.ipynb and use it to complete your climate analysis and data exploration.
Choose a start date and end date for your trip. Make sure that your vacation range is approximately 3-15 days total.
Use SQLAlchemy create_engine to connect to your sqlite database.
Use SQLAlchemy automap_base() to reflect your tables into classes and save a reference to those classes called Station and Measurement.

In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import pandas as pd
import matplotlib
import numpy as np
from sqlalchemy import func


import matplotlib.pyplot as plt

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [3]:
Base = automap_base()

In [4]:
Base.prepare(engine, reflect=True)

In [ ]:
Base.classes.keys()

In [5]:
Station = Base.classes.Station

In [15]:
Station.__dict__

mappingproxy({'__doc__': None,
              '__init__': <function __init__>,
              '__mapper__': <Mapper at 0x1071c8cf8; Station>,
              '__module__': 'sqlalchemy.ext.automap',
              '__table__': Table('Station', MetaData(bind=None), Column('station', VARCHAR(), table=<Station>, primary_key=True, nullable=False), Column('name', VARCHAR(), table=<Station>), Column('latitude', FLOAT(), table=<Station>), Column('longitude', FLOAT(), table=<Station>), Column('elevation', FLOAT(), table=<Station>), schema=None),
              '_sa_class_manager': <ClassManager of <class 'sqlalchemy.ext.automap.Station'> at 1071bbaf0>,
              'elevation': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x1071bbeb8>,
              'latitude': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x1071bbd58>,
              'longitude': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x1071bbe08>,
              'name': <sqlalchemy.orm.attributes.InstrumentedAttribute at 0x

In [6]:
Measurement = Base.classes.Measurement

In [7]:
session = Session(engine)

Precipitation Analysis


Design a query to retrieve the last 12 months of precipitation data.
Select only the date and prcp values.
Load the query results into a Pandas DataFrame and set the index to the date column.
Plot the results using the DataFrame plot method.

In [8]:
session.query(Measurement.date,Measurement.prcp).order_by(Measurement.date.desc()).first()

('2017-08-23', 0.0)

In [9]:
results = session.query(Measurement.date,Measurement.prcp).filter(Measurement.date>'2016-08-23')\
                                                        .order_by(Measurement.date).all()
results

[('2016-08-24', 0.08),
 ('2016-08-24', 2.15),
 ('2016-08-24', 2.28),
 ('2016-08-24', 0.0),
 ('2016-08-24', 1.22),
 ('2016-08-24', 2.15),
 ('2016-08-24', 1.45),
 ('2016-08-25', 0.08),
 ('2016-08-25', 0.08),
 ('2016-08-25', 0.0),
 ('2016-08-25', 0.0),
 ('2016-08-25', 0.21),
 ('2016-08-25', 0.06),
 ('2016-08-25', 0.11),
 ('2016-08-26', 0.0),
 ('2016-08-26', 0.03),
 ('2016-08-26', 0.02),
 ('2016-08-26', 0.04),
 ('2016-08-26', 0.0),
 ('2016-08-26', 0.01),
 ('2016-08-27', 0.0),
 ('2016-08-27', 0.18),
 ('2016-08-27', 0.02),
 ('2016-08-27', 0.0),
 ('2016-08-27', 0.12),
 ('2016-08-27', 0.0),
 ('2016-08-28', 0.01),
 ('2016-08-28', 0.14),
 ('2016-08-28', 0.14),
 ('2016-08-28', 0.14),
 ('2016-08-28', 0.6),
 ('2016-08-28', 2.07),
 ('2016-08-29', 0.0),
 ('2016-08-29', 0.17),
 ('2016-08-29', 0.04),
 ('2016-08-29', 0.0),
 ('2016-08-29', 0.0),
 ('2016-08-29', 0.35),
 ('2016-08-29', 0.9),
 ('2016-08-30', 0.0),
 ('2016-08-30', 0.0),
 ('2016-08-30', 0.02),
 ('2016-08-30', 0.0),
 ('2016-08-30', 0.0),
 ('20

In [ ]:
data_df = pd.DataFrame(results, columns=['date','prcp'])

In [ ]:
data_df.set_index('date', inplace=True, )

In [ ]:
data_df

In [ ]:
data_df.plot()
#plt.xticks(np.arange(9), ('2016-08-29', '2017-05-06', '2017-01-21', '2016-11-07', '2016-09-14'\
#                         ,'2016-11-19','2017-08-23','2017-05-05','2017-02-01'),rotation = 45)
plt.show()

Station Analysis


Design a query to calculate the total number of stations.

Design a query to find the most active stations.


List the stations and observation counts in descending order
Which station has the highest number of observations?



Design a query to retrieve the last 12 months of temperature observation data (tobs).


Filter by the station with the highest number of observations.
Plot the results as a histogram with bins=12.

In [ ]:
#Design a query to calculate the total number of stations.
session.query(Measurement.station).group_by(Measurement.station).count()

In [ ]:
#Design a query to find the most active stations.
# List the stations and observation counts in descending order
# Which station has the highest number of observations?
session.query(Measurement.station,func.count(Measurement.date)).group_by(Measurement.station)\
                                    .order_by(func.count(Measurement.date).desc()).all()

In [ ]:
#checking for most recent date in database to determine how far a year is
session.query(Measurement.station,func.count(Measurement.date)).group_by(Measurement.station)\
                .order_by(func.count(Measurement.date).desc()).first()

In [ ]:
#Design a query to retrieve the last 12 months of temperature observation data (tobs).
#Filter by the station with the highest number of observations.
weather_query = session.query(Measurement.date,Measurement.tobs)\
                        .filter(Measurement.station == 'USC00519281')\
                        .filter(Measurement.date>'2016-08-23').all()
weather_db = pd.DataFrame(weather_query,columns=['date','tobs'])
weather_db.set_index('date', inplace=True, )

In [ ]:
#Plot the results as a histogram with bins=12.
plt.hist(weather_db,bins =12);

Temperature Analysis


Write a function called calc_temps that will accept a start date and end date in the format %Y-%m-%d and return the minimum, average, and maximum temperatures for that range of dates.
Use the calc_temps function to calculate the min, avg, and max temperatures for your trip using the matching dates from the previous year (i.e. use "2017-01-01" if your trip start date was "2018-01-01")

Plot the min, avg, and max temperature from your previous query as a bar chart.


Use the average temperature as the bar height.
Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr).

In [ ]:
#Write a function called calc_temps that will accept a start date and end date in the format %Y-%m-%d and 
#return the minimum, average, and maximum temperatures for that range of dates.
#Use the calc_temps function to calculate the min, avg, and max temperatures for your trip using 
#the matching dates from the previous year (i.e. use "2017-01-01" if your trip start date was "2018-01-01")


def calc_temps(start_date='2016-08-23',end_date = '2017-08-23'):
    results = session.query(Measurement.tobs)\
                        .filter(Measurement.date<end_date)\
                        .filter(Measurement.date>start_date)\
                        .order_by(Measurement.tobs.desc()).all()
    
    min_temp = results[-1][0]
    avg_temp = np.mean(results)
    max_temp = results[0][0]
    
    return(min_temp,avg_temp,max_temp)
    

In [ ]:
# Plot the min, avg, and max temperature from your previous query as a bar chart.


# Use the average temperature as the bar height.
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr).


min_temp, avg_temp, max_temp = calc_temps('2016-08-23','2017-08-23')

plt.bar(1,avg_temp,yerr=(max_temp - min_temp))
plt.xlim(0,2);
plt.ylabel('Temp (F)');